### 導入模型所需之檔案

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import  metrics
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
data_path =  'C:\\Users\\User\\Desktop\\kaggle ML 期中考檔案\\'
train = pd.read_csv(data_path + 'train_data.csv')
test = pd.read_csv(data_path + 'test_features.csv')


### DataFrame測試集和訓練集並將'poi'設為目標值

In [2]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)
Target=train['poi']

### 移出不需要之特徵，將測試集和訓練集合併

In [3]:
df_train=train.drop(['name','poi','email_address'],axis=1)
df_test=test.drop(['name','email_address'],axis=1)
df_all = pd.concat([df_train, df_test])

In [4]:
df_all.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


### 將訓練集資料最大最小化並觀察特徵重要性

In [5]:
MMEncoder = MinMaxScaler()
for c in df_train.columns:
    df_train[c] = df_train[c].fillna(-1)
    df_train[c] = MMEncoder.fit_transform(df_train[c].values.reshape(-1, 1))
df_train.head()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,0.21875,0.03342,0.000000,0.0,0.576277,0.205238,0.001322,0.081285,0.012136,0.0,0.314261,0.016877,0.186182,0.927035,0.378523,0.191034,0.059802,0.004877,0.459506
1,0.70000,0.03342,1.000000,0.0,0.560434,0.128241,0.007586,0.168242,0.075243,0.0,0.373146,0.002135,0.463610,0.927035,1.000000,0.451193,0.239472,0.083843,0.531751
2,0.02500,0.03342,0.998811,0.0,0.047292,0.100038,0.002784,0.026465,0.036408,0.0,0.000000,0.151869,0.058884,0.927035,0.190635,0.020318,0.014917,0.019350,0.051628
3,0.10000,0.03342,1.000000,0.0,0.000000,0.516406,0.000000,0.000000,0.000000,0.0,0.117128,0.087599,0.066729,0.927035,0.201849,0.000000,0.000000,0.025614,0.020937
4,0.15625,0.03342,0.925094,0.0,0.000000,0.156576,0.010091,0.378072,0.063107,0.0,0.072939,0.000047,0.008538,0.927035,0.216142,0.483436,0.171551,0.015829,0.003461


In [6]:
estimator = RandomForestClassifier()
estimator.fit(df_train.values, Target)
feats = pd.Series(data=estimator.feature_importances_, index=df_train.columns)
feats = feats.sort_values(ascending=False)
feats

exercised_stock_options      0.135235
other                        0.117070
expenses                     0.113052
total_stock_value            0.098839
bonus                        0.077357
salary                       0.072831
restricted_stock             0.053496
total_payments               0.048433
from_messages                0.048375
long_term_incentive          0.043901
shared_receipt_with_poi      0.038413
deferred_income              0.037730
from_poi_to_this_person      0.036469
from_this_person_to_poi      0.034006
to_messages                  0.028940
deferral_payments            0.013212
loan_advances                0.002625
director_fees                0.000015
restricted_stock_deferred    0.000000
dtype: float64

### 由特徵重要性可知倒數四個欄位對預測值相關性低，將他們移除。

In [7]:
df_num=df_all.drop(['loan_advances','director_fees','restricted_stock_deferred','deferral_payments'],axis=1)
df_num['deferred_income']=abs(df_num['deferred_income'])
df_num.head()


,bonus,deferred_income,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,3504386.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,4167.0,1624396.0,22884.0,39.0,13.0,14.0,NaN,1573324.0,869220.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,118134.0,NaN,NaN,NaN,602671.0,907502.0,985032.0,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,262500.0,NaN,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,2188.0,2598.0,1639297.0,126027.0


### 檢查缺失值比率

In [8]:
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(100))
na_check(df_all)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_this_person_to_poi,41.095890
from_poi_to_this_person,41.095890
from_messages,41.095890


### 填補缺失值

In [9]:
# 部分欄位缺值補 'median'
median_cols = ['deferred_income','long_term_incentive']
for col in median_cols:
    df_num[col] = df_num[col].fillna(df_num[col].median())
# 部分欄位缺值補 'mean'
mean_cols = ['bonus','salary','expenses','exercised_stock_options','restricted_stock',
             'total_payments','total_stock_value','other','to_messages','shared_receipt_with_poi',
             'from_this_person_to_poi','from_poi_to_this_person','from_messages']
for col in mean_cols:
    df_num[col] = df_num[col].fillna(df_num[col].mean())  
# 特徵組合
df_num['Total poi']=(df_num['from_this_person_to_poi']+df_num['from_poi_to_this_person'])/2
df_num['Total message']=(df_num['to_messages']+df_num['from_messages'])/2
df_num['Total stock']=df_num['exercised_stock_options']**2
df_num['Total stock2']=df_num['exercised_stock_options']*df_num['total_stock_value']
df_num['Total stock3']=df_num['total_stock_value']**2

### 確認缺失值比率為0

In [10]:
na_check(df_num)

,Missing Ratio


### 將資料最大最小化，並檢查有無缺漏

In [11]:
MMEncoder = MinMaxScaler()
for c in df_num.columns:
    df_num[c] = MMEncoder.fit_transform(df_num[c].values.reshape(-1, 1))
df_num.head()

,bonus,deferred_income,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,Total poi,Total message,Total stock,Total stock2,Total stock3
0,0.017271,0.125162,0.063481,0.008940,0.000418,0.079545,0.006568,0.031944,0.004098,0.040269,0.015734,0.156188,0.056189,0.001629,0.051977,0.048988,0.043872,0.004031,0.003294,2.691574e-03
1,0.056850,0.005679,0.061736,0.005575,0.006687,0.166667,0.049261,0.038198,0.000518,0.071078,0.041596,0.369632,0.236549,0.028019,0.060149,0.125666,0.188500,0.003812,0.003708,3.606378e-03
2,0.001336,0.000119,0.005200,0.004343,0.001881,0.024621,0.022989,0.007694,0.036874,0.026133,0.007915,0.016126,0.011132,0.006466,0.005840,0.028754,0.009978,0.000027,0.000030,3.293083e-05
3,0.007505,0.005679,0.019193,0.022538,0.041571,0.122908,0.067705,0.011010,0.021269,0.027004,0.008382,0.212804,0.133638,0.008559,0.002368,0.113022,0.134375,0.000369,0.000044,5.134885e-06
4,0.012131,0.009348,0.019193,0.006814,0.009195,0.376894,0.041051,0.006317,0.000011,0.020542,0.008977,0.396086,0.168367,0.005290,0.000391,0.238552,0.137427,0.000369,0.000006,7.973304e-08


In [12]:
print(df_num.shape)

(146, 20)


### 將前述轉換完畢資料  , 重新切成 train_X, test_X

In [13]:
train_num = Target.shape[0]
train_X = df_num[:train_num]
test_X = df_num[train_num:]

### 使用二種模型 :  隨機森林 /梯度提升機 , 參數使用 Random Search 尋找(這邊不顯示程式碼)
### 訓練模型比較交叉驗證的分數，並帶入模型到測試集中

In [14]:
# 設定模型與模型參數
rf= RandomForestClassifier(max_features=5,max_depth=6,n_estimators=29,random_state=10)

# 使用 Train 資料訓練模型
rf.fit(train_X,Target)
rf_pred = rf.predict_proba(test_X)[:, 1]
cross_val_score(rf, train_X,Target, cv=5).mean()

0.9023715415019762

In [15]:
# 設定模型與模型參數
GDBT=GradientBoostingClassifier(n_estimators=40,learning_rate=0.05, min_samples_split=10,
                                min_samples_leaf=10,max_depth=3, subsample=0.8,random_state=10)
# 使用 Train 資料訓練模型
GDBT.fit(train_X,Target)
GDBT_pred = GDBT.predict_proba(test_X)[:, 1]
cross_val_score(GDBT, train_X,Target, cv=5).mean()

0.8853754940711462

### 儘管隨機森林在交叉驗證的分數較高，
### 但在經過好幾次的Kaggle分數比較後，發現GDBT效果較好，送出GDBT模型

In [16]:
submit = test[['name']]
submit['poi']=GDBT_pred

submit.head()
submit.to_csv('C:\\Users\\User\\Desktop\\kaggle ML 期中考檔案\\kaggle final.csv',index=False)

<ipython-input-16-317ef32898b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['poi']=GDBT_pred
